In [8]:
import os
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
from rapidfuzz import process, fuzz
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows


# Build engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)


#<---Formatting Lookup Table--->
def fmt_date(x):
    # Works whether x is Timestamp, date, or str/NaN
    try:
        if pd.isna(x): return ''
        return pd.to_datetime(x).strftime('%Y-%m-%d')
    except Exception:
        return str(x)

#<---Item Listing Fuzyy Search--->
# Load CSV
Itemlisting = pd.read_csv("Item Listing.csv", dtype=str, encoding_errors="ignore").fillna("")
# Keep only the text after the first ":" in Item
Itemlisting["Item"] = Itemlisting["Item"].str.split(":", n=1).str[-1].str.strip()

Itemlisting.columns = Itemlisting.columns.str.strip()

def normalize(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]", "", str(s)).lower()

# Normalized item column for searching
Itemlisting["__item_norm"] = Itemlisting["Item"].map(normalize)

def search_items(query: str, top_k: int = 15) -> pd.DataFrame:
    q = normalize(query)
    if not q:
        return pd.DataFrame(columns=Itemlisting.columns)

    # 1) Substring matches
    # Split query into words
    words = q.split()

    # Require all words to appear
    mask = Itemlisting["__item_norm"].apply(lambda name: all(w in name for w in words))
    direct = Itemlisting.loc[mask, Itemlisting.columns].copy()


    # 2) Fuzzy fallback 
    matches = process.extract(query, Itemlisting["Item"].tolist(), scorer=fuzz.WRatio, limit=top_k*2)
    fuzzy_rows = []
    seen = set()
    for _text, _score, idx in matches:
        if idx not in seen:
            seen.add(idx)
            fuzzy_rows.append(Itemlisting.iloc[idx])
    fuzzy = pd.DataFrame(fuzzy_rows)

    # 3) Combine results and dedupe by Item
    out = pd.concat([direct, fuzzy], ignore_index=True).drop_duplicates(subset=["Item"])
    return out.head(top_k)[["Item", "Description", 'Type', 'Cost', 'Price', 'Preferred Vendor']]


# Read entire table
df = pd.read_sql_table("wo_structured", con=engine, schema="public")

ERP_df= df[['Order Date', "Name", "P. O. #", "QB Num", "Item", 'Qty(-)', 
                              "Available + Pre-installed PO", 'Available', "Assigned Q'ty", 'On Hand - WIP', 'On Hand', 'On Sales Order', 'On PO', 'Reorder Pt (Min)', 'Available + On PO', 'Sales/Week', 'Recommended Restock Qty', 'Ship Date', 'Picked']]

ERP_df = ERP_df.copy()

ERP_df['SO_Status'] = np.where(
    ERP_df['Available'] < 0, 'Shortage', 'Available'
)

# LT Check

## 1. Transform NAV_Shipping

In [9]:
# ---- helpers ---------------------------------------------------------------
INCL_SPLIT = re.compile(r"\bincluding\b", re.IGNORECASE)
QTYX_RE = re.compile(r"^\s*(\d+)\s*x\s*(.+)\s*$", re.IGNORECASE) # "2x SSD-1TB"

def clean_space(s: str) -> str:
    if not isinstance(s, str):
        return ""
    # Normalize NBSP etc.
    return s.replace('\u00A0', ' ').replace('\u3000', ' ').strip()

def parse_description(desc: str) -> tuple[str, list[str]]:
    """
    Returns (parent_code, component_tokens[])
    e.g. "SEMIL-2047GC-CRL, including i9-13900E, 2x SSD-1TB"
    -> ("SEMIL-2047GC-CRL", ["i9-13900E", "2x SSD-1TB"])
    """
    s = clean_space(desc)
    parts = INCL_SPLIT.split(s, maxsplit=1)
    # parent part may have a trailing ", ..." — keep only before first comma
    parent = clean_space(parts[0].split(",")[0])
    comps = []
    if len(parts) > 1:
        comps = [clean_space(x) for x in parts[1].split(",") if clean_space(x)]
    return parent, comps

def parse_component_token(token: str) -> tuple[str, float]:
    """
    Parses a component token possibly with 'Nx ' prefix.
    Returns (item_code, qty_per_parent).
    """
    m = QTYX_RE.match(token)
    if m:
        qty = float(m.group(1))
        item = clean_space(m.group(2))
        return item, qty
    return clean_space(token), 1.0

def expand_preinstalled_row(row: pd.Series) -> pd.DataFrame:
    """
    For a single NAV row (Pre-installed), expand into multiple rows:
    - one row per component with Qty(+) multiplied by qty_per_parent
    - (optional) one row for the parent itself if you want to track it too
    """
    parent, tokens = parse_description(row.get("Description", ""))
    base_qty = float(row.get("Qty(+)", 0) or 0)
    parent_item = parent or clean_space(str(row.get("Item", "")))

    # Build component rows
    comp_rows = []
    for tok in tokens:
        item, qty_per = parse_component_token(tok)
        out = row.copy()
        out["Parent_Item"] = parent_item
        out["Item"] = item
        out["Qty_per_parent"] = qty_per
        out["Qty(+)"] = base_qty * qty_per  # multiply
        out["IsParent"] = False
        comp_rows.append(out)

    # Always include the parent row too (helps tracing/diagnostics).
    parent_row = row.copy()
    parent_row["Parent_Item"] = parent_item
    parent_row["Item"] = parent_item
    parent_row["Qty_per_parent"] = 1.0
    parent_row["IsParent"] = True

    if comp_rows:
        return pd.concat([pd.DataFrame(comp_rows), pd.DataFrame([parent_row])],
                         ignore_index=True)
    else:
        # No components in description → only the parent
        return pd.DataFrame([parent_row])

# ---- pipeline --------------------------------------------------------------

def expand_nav_preinstalled(NAV: pd.DataFrame) -> pd.DataFrame:
    NAV = NAV.copy()

    # --- sanity columns ---
    for col in ["Pre/Bare", "Qty(+)", "Item"]:
        if col not in NAV.columns:
            raise ValueError(f"NAV must contain '{col}' column.")
    if "Description" not in NAV.columns:
        NAV["Description"] = ""

    NAV["Description"] = NAV["Description"].astype(str).apply(clean_space)

    # --- split (make copies to avoid SettingWithCopyWarning) ---
    pre_mask = NAV["Pre/Bare"].astype(str).str.strip().str.casefold().eq("pre")
    nav_pre   = NAV.loc[pre_mask].copy()
    nav_other = NAV.loc[~pre_mask].copy()

    # --- expand all Pre rows ---
    expanded_parts = [expand_preinstalled_row(r) for _, r in nav_pre.iterrows()]
    expanded_pre = (
        pd.concat(expanded_parts, ignore_index=True)
        if expanded_parts else nav_pre.copy()
    )

    # --- ensure consistent columns via reindex (no chained assigns) ---
    needed_cols = list(NAV.columns) + ["Parent_Item", "Qty_per_parent", "IsParent"]

    expanded_pre = expanded_pre.reindex(columns=needed_cols, fill_value=pd.NA)
    nav_other    = nav_other.reindex(columns=needed_cols, fill_value=pd.NA)

    # --- set natural parent and flags for non-Pre ---
    nav_other.loc[:, "Parent_Item"]     = nav_other["Item"]
    nav_other.loc[:, "Qty_per_parent"]  = 1.0
    nav_other.loc[:, "IsParent"]        = True  # single line is its own parent

    # --- merge back ---
    expanded_all = pd.concat([expanded_pre, nav_other], ignore_index=True)

    # --- enforce dtypes ---
    expanded_all["Qty(+)"]         = pd.to_numeric(expanded_all["Qty(+)"], errors="coerce").fillna(0.0)
    expanded_all["Qty_per_parent"] = pd.to_numeric(expanded_all["Qty_per_parent"], errors="coerce").fillna(1.0)
    expanded_all["IsParent"]       = expanded_all["IsParent"].astype(bool)
    expanded_all["Date"] = pd.to_datetime(expanded_all["Ship Date"], errors="coerce") + pd.Timedelta(days=5)

    return expanded_all

## 2. Ledger

In [14]:
# -------------------------------
# Helpers
# -------------------------------

def _norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # standardize column names used here
    if "Ship Date" in df.columns:
        df["Ship Date"] = pd.to_datetime(df["Ship Date"], errors="coerce")
    if "Order Date" in df.columns:
        df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
    if "Arrive Date" in df.columns:
        df["Arrive Date"] = pd.to_datetime(df["Arrive Date"], errors="coerce")

    for c in ["Item"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

    # numeric fills
    for c in ["Qty(+)", "Qty(-)", "On Hand", "On Hand - WIP"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

    return df


def build_opening_stock(SO: pd.DataFrame) -> pd.DataFrame:
    """
    Build an opening-stock table per Item from SO snapshot columns.
    prefer_wip=True -> use 'On Hand - WIP' if present else 'On Hand'
    """
    src = SO.copy()
    col = "On Hand"
    if col not in src.columns:
        src[col] = 0.0

    # last non-null per item (or max—choice doesn't matter if consistent)
    stock = (
        src[["Item", col]]
        .dropna()
        .drop_duplicates(subset=["Item"], keep="last")
        .rename(columns={col: "Opening"})
    )
    return stock


def build_events(SO: pd.DataFrame, NAV_EXP: pd.DataFrame) -> pd.DataFrame:
    """
    Create a unified event table: IN from NAV_EXP, OUT from SO.
    Priority rule on same day: IN before OUT (so inbound covers same-day demand).
    """
    so = _norm_cols(SO)
    nav = _norm_cols(NAV_EXP)

    inbound = (
        nav.loc[nav["Qty(+)"] > 0, ["Date", "Item", "Qty(+)"]]
        .rename(columns={"Qty(+)": "Delta"})
        .assign(Kind="IN", Source="NAV")
    )

    outbound = (
        so.loc[so["Qty(-)"] > 0, ["Ship Date", "Item", "Qty(-)", "QB Num", "P. O. #", "Name"]]
        .rename(columns={"Ship Date": "Date", "Qty(-)": "Delta"})
        .assign(Kind="OUT", Source="SO")
    )

    # OUT is negative delta
    outbound["Delta"] = -outbound["Delta"]

    # Standardize columns before concat (missing cols become NaN)
    cols = ["Date", "Item", "Delta", "Kind", "Source", "QB Num", "P. O. #", "Name"]
    inbound  = inbound.reindex(columns=cols)
    outbound = outbound.reindex(columns=cols)

    events = pd.concat([inbound, outbound], ignore_index=True)
    # tie-breaker: IN before OUT on same date
    kind_order = {"IN": 0, "OUT": 1}
    events["kord"] = events["Kind"].map(kind_order).fillna(9).astype(int)

    events.sort_values(["Item", "Date", "kord"], inplace=True)
    events.drop(columns="kord", inplace=True)

    return events


def build_ledger(SO: pd.DataFrame, NAV_EXP: pd.DataFrame, prefer_wip=True) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      ledger: per-item time-phased projection
      item_summary: opening, min projected, first shortage date, ok flag
      violations: rows where projection < 0 (first neg per item marked)
    """
    so = _norm_cols(SO)
    nav = _norm_cols(NAV_EXP)

    stock = build_opening_stock(so)  # Item, Opening
    events = build_events(so, nav)                          # unified IN/OUT

    # Seed an "OPEN" row per item (so cumulative starts from Opening)
    open_rows = []
    today = pd.Timestamp.today().normalize()

    for item, opening in stock.itertuples(index=False):
        open_rows.append({
            "Date": today, "Item": item, "Delta": 0.0,
            "Kind": "OPEN", "Source": "Snapshot", "Opening": opening
        })
    open_df = pd.DataFrame(open_rows)

    # Attach Opening to all events for each item
    events = events.merge(stock, on="Item", how="left")
    events["Opening"] = events["Opening"].fillna(0.0)

    ledger = pd.concat([open_df, events], ignore_index=True, sort=False)

    # cumulative projection per item
    ledger["CumDelta"] = ledger.groupby("Item", sort=False)["Delta"].cumsum()
    # Projected NAV = Opening + CumDelta (OPEN row gives the base)
    ledger["Projected_NAV"] = ledger["Opening"] + ledger["CumDelta"]

    # For OUT rows, it’s useful to know before- and after-delta NAV
    ledger["NAV_before"] = np.where(
        ledger["Kind"].eq("OUT"),
        ledger["Projected_NAV"] - ledger["Delta"],  # since Delta negative
        np.nan
    )
    ledger["NAV_after"] = np.where(
        ledger["Kind"].eq("OUT"),
        ledger["Projected_NAV"],
        np.nan
    )

    # Item summary & violations
    item_min = ledger.groupby("Item", as_index=False)["Projected_NAV"].min().rename(columns={"Projected_NAV": "Min_Projected_NAV"})
    first_neg = (
        ledger.loc[ledger["Projected_NAV"] < 0]
        .sort_values(["Item", "Date"])
        .groupby("Item", as_index=False)
        .first()[["Item", "Date", "Projected_NAV"]]
        .rename(columns={"Date": "First_Shortage_Date", "Projected_NAV": "NAV_at_First_Shortage"})
    )

    item_summary = stock.merge(item_min, on="Item", how="outer").merge(first_neg, on="Item", how="left")
    item_summary["OK"] = item_summary["Min_Projected_NAV"].fillna(0) >= 0

    # Full violations table (all negative points)
    violations = ledger.loc[(ledger["Projected_NAV"]< 0) & (ledger['Date'] != '2099-12-31') & (ledger["Date"].notna())].copy()

    # nice ordering
    ledger.sort_values(["Item", "Date", "Kind"], inplace=True)
    item_summary.sort_values(["OK", "Min_Projected_NAV"], ascending=[True, True], inplace=True)

    return ledger, item_summary, violations


def compute_so_readiness(ledger: pd.DataFrame) -> pd.DataFrame:
    """
    Annotate OUT rows with whether they were covered on their Ship Date,
    and if not, the date inventory returns to non-negative (catch-up date).
    """
    out_rows = ledger.loc[ledger["Kind"].eq("OUT")].copy()

    # covered if NAV_before >= demand quantity (i.e., removing Delta wouldn't go negative)
    # equivalently, NAV_after >= 0
    out_rows["Covered_On_Date"] = out_rows["NAV_after"] >= 0

    # For rows not covered, find the next date where Projected_NAV becomes >= 0
    catchup_dates = []
    g = ledger.groupby("Item", sort=False)

    for idx, r in out_rows.loc[~out_rows["Covered_On_Date"]].iterrows():
        item = r["Item"]
        date = r["Date"]
        # find first row for this item on/after this date where Projected_NAV >= 0
        sub = g.get_group(item)
        sub = sub.loc[sub["Date"] >= date].sort_values("Date", kind="mergesort")
        hit = sub.loc[sub["Projected_NAV"] >= 0]
        catchup_dates.append((idx, hit["Date"].iloc[0] if not hit.empty else pd.NaT))

    catchup_map = {idx: dt for idx, dt in catchup_dates}
    out_rows["Covered_By_Date"] = out_rows.index.map(catchup_map).fillna(pd.NaT)

    return out_rows[[
        "Date", "Item", "Delta", "QB Num", "P. O. #", "Name",
        "NAV_before", "NAV_after", "Covered_On_Date", "Covered_By_Date"
    ]].sort_values(["Item", "Date"])

In [15]:
#SO
SO_INV = pd.read_sql_table("wo_structured", con=engine, schema="public")
SO = SO_INV[['Order Date', 'Ship Date', 'QB Num', "P. O. #", "Name",'Qty(+)', 'Qty(-)', 'Item', 'Pre/Bare', 'On Hand', "On Hand - WIP"]]
# SO.to_csv('open sales2.csv',index=False,columns =SO)

#"POD"
pod = pd.read_sql_table("Open_Purchase_Orders", con=engine, schema="public")
# pod.to_csv('open purchase2.csv', index=False)

#"NAV"
NAV= pd.read_sql_table("NT Shipping Schedule", con=engine, schema="public")
# NAV.to_csv('NAV1.csv', index=False)

NAV_EXP = expand_nav_preinstalled(NAV)

ledger, item_summary, violations = build_ledger(SO, NAV_EXP, prefer_wip=True)


In [94]:
SO.loc[SO['Item'] == 'Nuvo-9208VTC']

,Order Date,Ship Date,QB Num,P. O. #,Name,Qty(+),Qty(-),Item,Pre/Bare,On Hand,On Hand - WIP
419,2025-10-17,2025-11-18,SO-20251481,P96900,"CoastIPC, Inc.",0,13.0,Nuvo-9208VTC,Out,0.0,-13.0


In [12]:
Preinstalled_NAV = NAV_EXP.loc[NAV_EXP['Pre/Bare'] == 'Pre']
Preinstalled_NAV.to_excel(r"Preinstalled_NAV.xlsx", sheet_name="Sheet1", index=False)

In [21]:
ledger.loc[ledger['Item'] == 'GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0']

,Date,Item,Delta,Kind,Source,Opening,QB Num,P. O. #,Name,CumDelta,Projected_NAV,NAV_before,NAV_after
240,2025-11-04,GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0,0.0,OPEN,Snapshot,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN
741,2025-12-12,GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0,-3.0,OUT,SO,0.0,SO-20251515,4500047404,Rheinmetall Canada Inc.,-3.0,-3.0,0.0,-3.0
742,2099-12-31,GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0,-3.0,OUT,SO,0.0,SO-20251550,1016,"Acuitus Ag, Inc.",-6.0,-6.0,-3.0,-6.0


In [16]:
violations

,Date,Item,Delta,Kind,Source,Opening,QB Num,P. O. #,Name,CumDelta,Projected_NAV,NAV_before,NAV_after
718,2025-11-25,Extnd-mPCIeHS-BTWifi-WT-6218_Mod_Cbl-40CM_kits,-1.0,OUT,SO,0.0,SO-20251348,1034004597,FEV North America Inc.,-1.0,-1.0,0.0,-1.0
720,2025-11-25,Extnd-mPCIeHS_GPS-M800_Mod_Cbl-40CM_kits,-1.0,OUT,SO,0.0,SO-20251348,1034004597,FEV North America Inc.,-1.0,-1.0,0.0,-1.0
741,2025-12-12,GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0,-3.0,OUT,SO,0.0,SO-20251515,4500047404,Rheinmetall Canada Inc.,-3.0,-3.0,0.0,-3.0
745,2025-11-18,GC-RTX2000Ada-Nvidia,-5.0,OUT,SO,2.0,SO-20251333,08000,Monteris Medical,-5.0,-3.0,2.0,-3.0
746,2025-11-24,GC-RTX2000Ada-Nvidia,2.0,IN,NAV,2.0,NaN,NaN,NaN,-3.0,-1.0,NaN,NaN
811,2026-01-13,M.280-SSD-1TB-PCIe44-TLC5-PN,-10.0,OUT,SO,41.0,SO-20250641,PO064732,"FireFly Automatix, Inc.",-42.0,-1.0,9.0,-1.0
812,2026-02-13,M.280-SSD-1TB-PCIe44-TLC5-PN,-10.0,OUT,SO,41.0,SO-20250642,PO064732,"FireFly Automatix, Inc.",-52.0,-11.0,-1.0,-11.0
1146,2025-12-09,SEMIL-1748GC-10G-L4-BSK(EA),-1.0,OUT,SO,0.0,SO-20251390,455802,Navico Group Americas LLC,-1.0,-1.0,0.0,-1.0
1158,2026-02-20,SSD-1920GB-TLC-IL1,-2.0,OUT,SO,20.0,SO-20250332,PO208679,Noah Medical U.S.,-22.0,-2.0,0.0,-2.0
1159,2026-04-24,SSD-1920GB-TLC-IL1,-10.0,OUT,SO,20.0,SO-20250328,PO208679,Noah Medical U.S.,-32.0,-12.0,-2.0,-12.0


In [98]:
violations.to_excel(r"violations.xlsx", sheet_name="Sheet1", index=False)

## SO not has been assigned LT yet

In [2]:
# --- Your filter ---
assigned_mask = (
    (ERP_df["Ship Date"].dt.month.eq(7)  & ERP_df["Ship Date"].dt.day.eq(4)) |
    (ERP_df["Ship Date"].dt.month.eq(12) & ERP_df["Ship Date"].dt.day.eq(31))
)
Not_assgned_SO = ERP_df[assigned_mask].copy()

# --- Output path ---
output_path = r"Not_assigned_SO.xlsx"

# =======================
# 1) SAVE SHEET1 (ALL ROWS)
# =======================
# Find current first sheet's name, then replace just that sheet
_wb = load_workbook(output_path)
first_sheet_name = _wb.sheetnames[0]
_wb.close()

with pd.ExcelWriter(output_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    Not_assgned_SO.to_excel(writer, sheet_name=first_sheet_name, index=False)

# ======================= 
# 2) OPEN WORKBOOK & PREP STYLING
# =======================
from datetime import datetime
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl import load_workbook

wb = load_workbook(output_path)
ws = wb.worksheets[0] 

# Freeze the first row (header)
ws.freeze_panes = "A2"

# Build header -> column index map for Sheet1
wo_col = None
status_col = None
col_map = {}
for idx, cell in enumerate(ws[1], 1):  # header row
    col_map[cell.value] = idx
    if cell.value == "QB Num":
        wo_col = idx
    if cell.value == "SO_Status":
        status_col = idx

# Define fills & fonts
gray_fill   = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
white_fill  = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # for Recommended Restock Qty > 0
red_font    = Font(color="FF0000")
center_align = Alignment(horizontal="center", vertical="center")

# =======================
# 3) FORMAT SHEET1 (band by WO, red rows for Shortage, widths, alignment)
# =======================
current_wo = None
fill_toggle = False

if wo_col is not None and status_col is not None:
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        wo_cell = row[wo_col - 1]
        status_cell = row[status_col - 1]

        # Toggle background when WO changes
        if wo_cell.value != current_wo:
            current_wo = wo_cell.value
            fill_toggle = not fill_toggle

        # Apply background fill to row
        fill = gray_fill if fill_toggle else white_fill
        for c in row:
            c.fill = fill

        # If shortage, make entire row red font
        if status_cell.value == "Shortage":
            for c in row:
                c.font = red_font

# NEW: Highlight "Recommended Restock Qty" if > 0
target_col_name = "Recommended Restock Qty"
if target_col_name in col_map:
    col_idx = col_map[target_col_name]
    for (cell,) in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
        # Safely coerce to number
        try:
            val = float(cell.value)
        except (TypeError, ValueError):
            val = 0
        if val > 0:
            cell.fill = yellow_fill  # override banding fill for this cell only

# Set column widths (if they exist)
col_widths = {
    'Order Date': 15,
    "Item": 30,
    "Name": 25,
    "P. O. #": 15,
    "QB Num": 15,
    "Qty(-)": 10,
    "Available": 15,
    'Available + Pre-installed PO': 25,
    'On Hand - WIP': 20,
    'Reorder Pt (Min)': 15,
    'Recommended Restock Qty': 20,
    'On Sales Order': 15
}
for col_name, width in col_widths.items():
    if col_name in col_map:
        col_letter = ws.cell(row=1, column=col_map[col_name]).column_letter
        ws.column_dimensions[col_letter].width = width

# Center-align specific columns on Sheet1
for col_name in ["Qty", "Available + Pre-installed PO", "Available"]:
    if col_name in col_map:
        col_idx = col_map[col_name]
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
            for cell in row:
                cell.alignment = center_align

# Rename Sheet1 to today's date (e.g., "2025-10-05")
today_str = datetime.today().strftime("%Y-%m-%d")
ws.title = today_str

# =======================
# 4) SAVE + SUMMARY
# =======================
wb.save(output_path)

unique_wo_count = Not_assgned_SO["QB Num"].nunique() if "QB Num" in Not_assgned_SO.columns else 0
print(f"Number of unassigned WOs: {unique_wo_count}")
# print(f"Written:\n  - Sheet '{today_str}' (formerly Sheet1)\n  - Sheet 'Sheet2' (SO Entry Date == {target_date}) -> {len(today_only)} rows")

Number of unassigned WOs: 24


# Visualization

In [31]:

filtered_df = ERP_df[ERP_df['QB Num'] == 'SO-20251459']

# filtered_df = ERP_df[ERP_df['Customer'] == 'Aerflite Canada Inc.']


# Make Lead Time real datetimes
filtered_df = filtered_df.copy()
filtered_df['Ship Date'] = pd.to_datetime(filtered_df['Ship Date'], errors='coerce')


# Create styled object
styled = (
    filtered_df.style
    .map(lambda v: 'background-color: red; color: white' if v == 'Shortage' else '', subset='SO_Status')
    # General table & header styling
    .set_table_styles([
        # Header style
        {'selector': 'thead th',
         'props': [
             ('background-color', '#444'),
             ('color', 'white'),
             ('font-weight', 'bold'),
             ('text-align', 'center'),
             ('border', '1px solid #777'),
             ('padding', '6px 8px')
         ]},

        # Table body cells
        {'selector': 'td',
         'props': [
             ('color', 'white'),
             ('border', '1px solid #555'),
             ('text-align', 'center'),
             ('padding', '4px 6px')
         ]},

        # Highlight column header (Available + Pre-installed PO)
        {'selector': 'th.col' + str(filtered_df.columns.get_loc('Available + Pre-installed PO')),
         'props': [
             ('background-color', '#ffae42'),
             ('color', 'black'),
             ('font-weight', 'bold'),
             ('border', '1px solid #777'),
             ('min-width', '100px')
         ]},

        # Widen Lead Time column
        {'selector': 'th.col' + str(filtered_df.columns.get_loc('Ship Date')),
         'props': [
             ('min-width', '120px'),     # increase column width
             ('max-width', '150px')
         ]},

        # Optional — hover effect
        {'selector': 'tbody tr:hover',
         'props': [('background-color', '#333')]}
    ])
    .format({'Lead Time': fmt_date}, precision=0, na_rep='', thousands=',')
)

display(styled)




,Order Date,Name,P. O. #,QB Num,Item,Qty(-),Available + Pre-installed PO,Stock_Available,Assigned Q'ty,In Stock(Inventory),On Hand,On Sales Order,On PO,Reorder Pt (Min),Available + On PO,Sales/Week,Recommended Restock Qty,Ship Date,Picked,SO_Status
915,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,Nuvo-10108GC-10G,1,3,3,1,4,4,1,1,0,4,0,0,2025-11-10 00:00:00,No,Available
916,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,i7-14700,1,-6,-6,7,7,7,13,19,0,13,3,0,2025-11-10 00:00:00,No,Shortage
917,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,GC-RTX5070Ti-OC-MSI,1,-1,-1,1,0,0,1,1,0,0,0,0,2025-11-10 00:00:00,No,Shortage
918,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,DDR5-32GB-48-SM,2,58,58,32,103,103,45,0,50,58,7,0,2025-11-10 00:00:00,No,Available
919,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,M.280-SSD-2TB-PCIe44-TLC5ET-TD,1,15,15,29,45,45,30,4,10,19,3,0,2025-11-10 00:00:00,No,Available
920,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,mPCIeHS-WiFi-121N,1,5,5,35,52,52,47,0,15,5,2,5,2025-11-10 00:00:00,No,Available
921,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,Extnd-mPCIeHS,1,5,5,54,77,77,72,10,50,15,6,8,2025-11-10 00:00:00,No,Available
922,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,Win11IoT24-High,1,2,2,7,11,11,9,0,0,2,1,1,2025-11-10 00:00:00,No,Available
923,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,PCIe-PoE550X,3,7,7,3,10,10,3,0,10,7,1,0,2025-11-10 00:00:00,No,Available
924,2025/10/13,"LM3 Technologies, Inc.",1792-58-1,SO-20251459,AccsyBx-Cardholder-10108GC-5080,1,-3,-3,1,0,0,3,5,0,2,0,0,2025-11-10 00:00:00,No,Shortage


In [8]:
filtered_df = ERP_df[ERP_df['Product Number'] == 'NRU-120S-AGX32G'].sort_values(by='Lead Time',ascending=True)
filtered_df

,SO Entry Date,Customer,Customer PO,WO,Product Number,Qty,Available + Pre-installed PO,Stock_Available,On Hand,In Stock(Inventory),Assigned Q'ty,On Sales Order,On PO,Available + On PO,Sales/Week,Recommended Restock Qty,Lead Time,Picked,SO_Status,Reorder Pt (Min)
412,09/10/2025,IRD- International Road Dynamics Inc.,PO18173,SO-20251287,NRU-120S-AGX32G,1.0,-4.0,-4.0,0.0,0.0,4.0,4.0,4.0,0.0,0.5,2,2025-10-24,No,Shortage,0.0
827,10/01/2025,IRD- International Road Dynamics Inc.,PO18432,SO-20251410,NRU-120S-AGX32G,3.0,-4.0,-4.0,0.0,0.0,4.0,4.0,4.0,0.0,0.5,2,2025-10-29,No,Shortage,0.0


## Part Name Lookup

In [40]:
results = search_items("cblm12")
print(results.to_string(index=False))

                           Item                                                                                         Description           Type  Cost Price           Preferred Vendor
         Cbl-M12A10M-DB9M-180CM Cbl-M12A10M-DB9M-180CM..M12 A-Code 10P (male) to DB9 (male), Length: 180CM. Industrial computer ... Inventory Part 14.00 25.00 Neousys Technology Incorp.
    Cbl-M12A17M-2DB9M_OW2-180CM Cbl-M12A17M-2DB9M_OW2-180CM..M12 (17-pole-A-coded) to 2xDB9 (Male) and 1xopen wire 2P, Length: 1... Inventory Part 15.00 19.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM1 Cbl-M12A17M-VGA-180CM1..M12 (17-pole-A-coded) to VGA (male), Length : 180CM. Industrial computer... Inventory Part 16.00 20.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM2                        Cbl-M12A17M-VGA-180CM2..M12 (17-pole-A-coded) to VGA (male), Length : 180CM. Inventory Part 10.00 13.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM3 Cbl-M12A17M-VGA-180CM3..M12 (17-pole-A

## Items that can still be assigned although have negative available

In [8]:
filtered_df1 = ERP_df[(ERP_df['Stock_Available'] < 0) & (ERP_df['Available + Pre-installed PO'] > 0)]
unique_parts_df = filtered_df1.drop_duplicates(subset=['Product Number'])
unique_parts_df[['Product Number', 'Stock_Available', 'On PO', 'Available + Pre-installed PO']]

,Product Number,Stock_Available,On PO,Available + Pre-installed PO
76,M.2-LTE-7455,-58.0,85.0,27.0
77,Risr-M2B-mPCIe-SIMslot,-83.0,95.0,2.0
78,Cbl-MHF4-SMAF-30CM,-235.0,280.0,20.0
148,GC-Jetson-AGX32GB-Orin-Nvidia,-83.0,85.0,2.0
158,POC-330,-25.0,52.0,27.0
159,DDR3L-4GB-WT18-DL1,-27.0,52.0,25.0
160,mSATAHS-64GB-MLCWT-IK,-27.0,52.0,25.0
161,Win10IoT19-Entry,-27.0,52.0,25.0
189,DtC-M12-WP,-14.0,19.0,5.0
193,DDR4-32GB-WT32-SM,-40.0,98.0,14.0


## Vendor POD

In [11]:
NT_Shipping = pd.read_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\Daily Update\NTA_Shipping schedule_20251013.xlsx")
NT_Shipping.loc[NT_Shipping['Customer PO No.'] == 'POD-251418']

,Date,SO NO.,Ship to,Customer PO No.,Reference,Project Code,Model Name,Assemble Option,Qty,Remark,Ship Date,Description
165,2025-10-03,SO25100058,"Neousys Technology America, Inc.",POD-251418,NTA_INVENTORY_IRD,NaN,Nuvo-7006LP-PoE,No,4,NaN,2025-10-22 00:00:00,Intel® Coffee Lake 8th-Gen Core™ i fanless em...
166,2025-10-03,SO25100058,"Neousys Technology America, Inc.",POD-251418,NTA_INVENTORY_IRD,NaN,Dmpbr-Nuvo5000_7000,No,5,NaN,2025-10-22 00:00:00,Damping bracket assembly for Nuvo-5000/7000/90...
167,2025-10-03,SO25100058,"Neousys Technology America, Inc.",POD-251418,NTA_INVENTORY_IRD,NaN,NRU-120S-AGX32G,No,3,NaN,2025-10-22 00:00:00,"NRU-120S, including GC-Jetson-AGX-Xavier-Nvid..."
168,2025-10-03,SO25100058,"Neousys Technology America, Inc.",POD-251418,NTA_INVENTORY_IRD,NaN,AccsyBx-FAN-NRU-100,No,5,NaN,2025-10-22 00:00:00,"Fan kit for NRU-110V, NRU-120S, NRU-220S. It i..."


In [19]:
NT_Shipping.loc[NT_Shipping['Model Name'] == 'AccsyBx-Cardholder-10108GC-5080_70_70Ti']

,Date,SO NO.,Ship to,Customer PO No.,Reference,Project Code,Model Name,Assemble Option,Qty,Remark,Ship Date,Description
98,2025-10-13,SO25100195,"Neousys Technology America, Inc.",POD-251444,NTA_INVENTORY,NaN,AccsyBx-Cardholder-10108GC-5080_70_70Ti,No,5,NaN,2025-10-22 00:00:00,"Nuvo-10108GC GPU bracket kit for MSI RTX 5080,..."


In [34]:
filtered_df = ERP_df[ERP_df['Product Number'] == 'i9-13900'].sort_values(by='Lead Time',ascending=True)
filtered_df

,SO Entry Date,Customer,Customer PO,WO,Product Number,Qty,Available + Pre-installed PO,Stock_Available,On Hand,In Stock(Inventory),Assigned Q'ty,On Sales Order,On PO,Reorder Pt (Min),Sales/Week,Recommended Restock Qty,Lead Time,Picked,SO_Status
549,09/18/2025,Boston Scientific,7000615510,SO-20251324,i9-13900,1.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-15,No,Shortage
388,09/03/2025,Boston Scientific,7000610874,SO-20251250,i9-13900,1.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-20,No,Shortage
463,09/09/2025,Boston Scientific,7000612884,SO-20251283,i9-13900,3.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-21,No,Shortage
564,09/18/2025,Boston Scientific,7000615802,SO-20251325,i9-13900,4.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-24,No,Shortage
759,09/30/2025,Boston Scientific,7000618386,SO-20251393,i9-13900,1.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-29,No,Shortage
774,09/30/2025,Boston Scientific,7000619548,SO-20251394,i9-13900,2.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-29,No,Shortage
789,09/30/2025,Boston Scientific,7000620183,SO-20251395,i9-13900,1.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-29,No,Shortage
140,06/06/2025,"Applied Intuition, Inc.",AI-001279,SO-20250800,i9-13900,2.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-31,Picked,Shortage
154,07/15/2025,"Applied Intuition, Inc.","AI-001620, AI-001621",SO-20250995,i9-13900,4.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-31,Picked,Shortage
839,09/30/2025,F.K. MACHINERY LTD.,4017695,SO-20251400,i9-13900,1.0,-1.0,-1.0,21.0,15.0,20.0,22.0,4.0,0.0,6.1,0.0,2025-10-31,No,Shortage


In [17]:
pod = df = pd.read_sql_table("Open_Purchase_Orders", con=engine, schema="public")
pod.loc[pod['Item'] == 'AccsyBx-Cardholder-10108GC-5080']

,Order Date,QB Num,Name,Source Name,Item,Deliv Date,Qty(+)


In [96]:
pod.loc[pod['QB Num'] == 'Adapter-Active-DP-HDMI-BENFEI']

,Order Date,QB Num,Name,Source Name,Item,Deliv Date,Qty(+)


In [13]:
# Filter rows where Name is not 'Neousys Technology Incorp.'
filtered = pod[pod['Name'] != 'Neousys Technology Incorp.']

# Group by Item and sum Qty(+)
result = (
    filtered.groupby('Item', as_index=False)['Qty(+)']
    .sum()
)

lookup = (
    result[['Item', 'Qty(+)']]
    .drop_duplicates(subset=['Item'])         # ensures uniqueness
    .set_index('Item')['Qty(+)'] # Series: index = part_number
)

# 3) Map onto df (row count stays the same)
ERP_df['Qty(+)'] = ERP_df['Product Number'].map(lookup)
ERP_df



,Customer,Customer PO,WO,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status,Qty(+)
0,"CoastIPC, Inc.",P96695,EO-20250002,RGS-8805GC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-31,No,Available,NaN
1,Xanthon LLC,X110992,SO-20240315,POC-715,1.0,30.0,30.0,36.0,39.0,42.0,12.0,0.0,2025-12-19,No,Available,NaN
2,Xanthon LLC,X110992,SO-20240315,DDR5-16GB-WT48-IK,1.0,47.0,47.0,47.0,138.0,182.0,135.0,0.0,2025-12-19,No,Available,NaN
3,Xanthon LLC,X110992,SO-20240315,M.280-SSD-1TB-PCIe4-TLCWT-IK1,1.0,51.0,51.0,51.0,52.0,52.0,1.0,0.0,2025-12-19,No,Available,NaN
4,Xanthon LLC,X110992,SO-20240315,AccsyBx-FAN-POC-700,1.0,69.0,69.0,69.0,70.0,70.0,1.0,0.0,2025-12-19,No,Available,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,Noah Medical U.S.,PO209761,SO-20251423,Extnd-mPCIeHS,4.0,28.0,28.0,26.0,67.0,84.0,56.0,0.0,2025-12-31,No,Available,NaN
1042,Noah Medical U.S.,PO209761,SO-20251423,Cbl-MHF-RP_SMAF-30CM,8.0,26.0,26.0,22.0,92.0,118.0,92.0,25.0,2025-12-31,No,Available,NaN
1043,Noah Medical U.S.,PO209761,SO-20251423,Ant-RP_SMAM-WiFi-196MM1,8.0,13.0,-237.0,0.0,49.0,123.0,360.0,252.0,2025-12-31,No,Shortage,250.0
1044,Noah Medical U.S.,PO209761,SO-20251424,GC-RTXA4500-PNY,10.0,6.0,6.0,37.0,37.0,37.0,31.0,0.0,2025-12-31,No,Available,NaN


### Incoming QTY (assigned to WO) + Stock_Available <= 0

In [14]:
filtered_df1 = ERP_df[(ERP_df['Qty(+)']+ERP_df['Stock_Available']) <= 0].drop_duplicates(subset=['Product Number']).drop(['Customer', 'Customer PO', 'WO'], axis=1)
filtered_df1

,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status,Qty(+)
5,E-2278GE,5.0,-5.0,-6.0,0.0,19.0,29.0,35.0,6.0,2025-10-24,Picked,Shortage,1.0
98,M.280-SSD-1TB-PCIe44-TLC5-PN,10.0,-11.0,-24.0,0.0,29.0,39.0,63.0,43.0,2025-10-03,Picked,Shortage,13.0
99,Cbl-M12S4F-OW4-180CM1,10.0,0.0,-40.0,0.0,0.0,10.0,50.0,40.0,2025-10-03,Picked,Shortage,40.0
143,i7-9700E,20.0,-1.0,-21.0,0.0,-1.0,19.0,40.0,30.0,2025-10-01,Picked,Shortage,20.0
156,NRU-230V-AWP,2.0,0.0,-16.0,0.0,3.0,5.0,21.0,20.0,2025-08-06,Picked,Shortage,16.0
160,AC-IMX490-H120,1.0,0.0,-1.0,0.0,0.0,1.0,2.0,1.0,2025-08-06,Picked,Shortage,1.0
161,Cbl-M12A8M-2U2TA-180CM2,3.0,0.0,-4.0,0.0,0.0,3.0,7.0,4.0,2025-08-06,Picked,Shortage,4.0
165,SSD-2TB-TLC-IK,25.0,0.0,-50.0,0.0,25.0,25.0,75.0,50.0,2025-10-08,No,Shortage,50.0
167,Cblkit-BP-NRU-230V-AWP,25.0,0.0,-53.0,0.0,25.0,26.0,79.0,57.0,2025-10-08,No,Shortage,53.0
179,Ant-RP_SMAM-WiFi-108MM,8.0,0.0,-20.0,0.0,14.0,22.0,42.0,40.0,2025-09-11,Picked,Shortage,20.0


In [4]:
ERP_df.head(10)

,Order Date,Name,P. O. #,QB Num,Item,Qty(-),Available + Pre-installed PO,Available,Assigned Q'ty,On Hand - WIP,On Hand,On Sales Order,On PO,Reorder Pt (Min),Available + On PO,Sales/Week,Recommended Restock Qty,Ship Date,Picked,SO_Status
0,2025-08-22,"CoastIPC, Inc.",P96695,EO-20250002,RGS-8805GC,1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.1,2,2099-12-31,No,Shortage
1,2024-03-13,Xanthon LLC,X110992,SO-20240315,POC-715,1.0,26.0,26.0,1.0,29.0,29.0,3.0,0.0,4.0,26.0,1.5,0,2025-12-19,No,Available
2,2024-03-13,Xanthon LLC,X110992,SO-20240315,DDR5-16GB-WT48-IK,1.0,40.0,40.0,8.0,47.0,48.0,8.0,0.0,0.0,40.0,1.7,0,2025-12-19,No,Available
3,2024-03-13,Xanthon LLC,X110992,SO-20240315,M.280-SSD-1TB-PCIe4-TLCWT-IK1,1.0,37.0,35.0,8.0,48.0,49.0,14.0,2.0,0.0,37.0,1.1,0,2025-12-19,No,Available
4,2024-03-13,Xanthon LLC,X110992,SO-20240315,AccsyBx-FAN-POC-700,1.0,69.0,69.0,1.0,70.0,70.0,1.0,0.0,0.0,69.0,0.2,0,2025-12-19,No,Available
5,2025-03-05,Noah Medical U.S.,PO208679,SO-20250326,E-2278GE,5.0,-23.0,-25.0,35.0,30.0,40.0,65.0,5.0,5.0,-20.0,1.7,27,2026-01-09,Picked,Shortage
6,2025-03-05,Noah Medical U.S.,PO208679,SO-20250326,DDR4-32GB-32-SM,10.0,-10.0,-10.0,62.0,92.0,112.0,122.0,20.0,50.0,10.0,2.4,0,2026-01-09,Picked,Shortage
7,2025-03-05,Noah Medical U.S.,PO208679,SO-20250326,SSD-1920GB-TLC-IL1,10.0,-102.0,-102.0,62.0,0.0,20.0,122.0,0.0,0.0,-102.0,0.9,106,2026-01-09,Picked,Shortage
8,2025-03-05,Noah Medical U.S.,PO208679,SO-20250326,mPCIeHS-WiFi-121N,5.0,-17.0,-17.0,31.0,36.0,46.0,63.0,0.0,15.0,-17.0,2.6,28,2026-01-09,Picked,Shortage
9,2025-03-05,Noah Medical U.S.,PO208679,SO-20250326,Extnd-mPCIeHS,5.0,-10.0,-10.0,42.0,56.0,66.0,76.0,0.0,50.0,-10.0,5.8,34,2026-01-09,Picked,Shortage


## Ready to Pick (Ins Stock > Required Qty)

In [7]:
# True for WOs where ALL lines have In Stock > 0 and not been picked
wo_ok = (
    ERP_df.groupby("QB Num")
    .apply(lambda g: (g["On Hand - WIP"] >= g['Qty(-)']).all() and (g["Picked"].eq("No").all()))
)


# 1) list of ready WOs
ready_wos = wo_ok[wo_ok].index.tolist()


# 2) full rows for those WOs
ready_df = ERP_df[ERP_df["QB Num"].isin(ready_wos)].copy()

print("Ready WOs:", ready_wos)


ready_df.to_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx", sheet_name="Sheet2", index=False)

Ready WOs: ['SO-20240315', 'SO-20251248', 'SO-20251311', 'SO-20251343', 'SO-20251345', 'SO-20251359', 'SO-20251366', 'SO-20251405', 'SO-20251412', 'SO-20251445', 'SO-20251455', 'SO-20251456', 'SO-20251476', 'SO-20251487', 'SO-20251490', 'SO-20251491', 'SO-20251498', 'SO-20251499', 'SO-20251502', 'SO-20251517', 'SO-20251537', 'SO-20251543', 'SO-20251545', 'SO-20251548', 'SO-20251551', 'SO-20251552', 'SO-20251554']


C:\Users\Admin\AppData\Local\Temp\ipykernel_22720\441336993.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["On Hand - WIP"] >= g['Qty(-)']).all() and (g["Picked"].eq("No").all()))


## Long Aged Items

In [ ]:
import json, re
import pandas as pd

Long_aged = pd.read_sql_table("Long_Aged_Items", con=engine, schema="public")
word_files_df = pd.read_sql_table("word_file_log", con=engine, schema="public")

# --- normalize and mappings ---
mappings = {
    'GC-J-A64GB-O-Industrial-Nvidia': 'GC-Jetson-AGX64GB-Orin-Industrial-Nvidia-JetPack-6.0',
    'GC-Jetson-AGX64GB-Orin-Nvidia': 'GC-Jetson-AGX64GB-Orin-Nvidia-JetPack-6.0',
    # ... (keep rest)
}

def normalize(s: str) -> str:
    """Lowercase and remove all non-alphanumeric chars for robust matching."""
    return re.sub(r'[^a-z0-9]', '', str(s or '').strip().lower())

def norm_map_value(s: str) -> str:
    """Apply mappings then normalize."""
    s2 = mappings.get(s, s)
    return normalize(s2)

# 1️⃣ Build normalized lookup set from Long Aged Items
items_norm = { norm_map_value(x) for x in Long_aged["Item"].dropna().astype(str) }

# 2️⃣ Extract product numbers from JSON/list
def extract_numbers(cell):
    if cell is None:
        return []
    data = cell
    if isinstance(cell, str):
        try:
            data = json.loads(cell)
        except Exception:
            return []
    if isinstance(data, list):
        return [d.get("product_number") or d.get("part_number") for d in data if isinstance(d, dict)]
    return []

# 3️⃣ Check if a row contains any long-aged item
def contains_any_product(cell) -> bool:
    pns = extract_numbers(cell)
    return any(norm_map_value(pn) in items_norm for pn in pns)

# 4️⃣ Create a mask for matched rows
mask = word_files_df["product_details"].apply(contains_any_product)

# 5️⃣ Create the filtered DataFrame (keeping full product_details)
df_filtered = word_files_df[mask].copy()

# 6️⃣ Also show which specific products matched
def matched_list(cell):
    pns = extract_numbers(cell)
    return [pn for pn in pns if norm_map_value(pn) in items_norm]

df_filtered["matched_items"] = df_filtered["product_details"].apply(matched_list)

import json
import pandas as pd

# --- helper: get first product_number from product_details cell ---
def first_product_number(cell):
    if cell is None:
        return None
    data = cell
    if isinstance(cell, str):
        try:
            data = json.loads(cell)
        except Exception:
            return None
    if isinstance(data, list) and data and isinstance(data[0], dict):
        return data[0].get("product_number") or data[0].get("part_number")
    return None

# df_filtered already has: file_name, product_details, matched_items
df_keyed = df_filtered.copy()
df_keyed["first_product_number"] = df_keyed["product_details"].apply(first_product_number)

# 1) Flat mapping: key -> file_name + matched_items (as list)
flat = df_keyed.loc[df_keyed["first_product_number"].notna(),
                    ["first_product_number", "file_name", "matched_items", "product_details"]]

# (Optional) make matched_items a comma string for readability
flat["matched_items_str"] = flat["matched_items"].apply(lambda xs: ", ".join(xs) if isinstance(xs, list) else "")

# 2) Per-key summary: aggregate all files and all matched items under each first_product_number
summary = (
    flat.groupby("first_product_number", as_index=False)
        .agg(
            files=("file_name", list),
            matched_items=("matched_items", lambda lists: sorted({x for sub in lists for x in (sub or [])}))
        )
)

# Nice previews
# print("=== Flat mapping (first 10) ===")
# display(flat[["first_product_number", "file_name", "matched_items_str"]])
# Aged_Items_Usage = flat[["first_product_number", "file_name", "matched_items_str"]]
# Aged_Items_Usage.to_excel("Aged_Items_Usage.xlsx")

print("=== Per-key summary (first 10) ===")
display(summary)
summary.to_excel("Aged_Items_Usage.xlsx")